## TP Final - Optimizacion

In [1]:
import cplex

In [2]:
TOLERANCE =10e-6 

class Orden:
    def __init__(self):
        self.id = 0
        self.beneficio = 0
        self.trabajadores_necesarios = 0
    
    def load(self, row):
        self.id = int(row[0])
        self.beneficio = int(row[1])
        self.trabajadores_necesarios = int(row[2])
        

class FieldWorkAssignment:
    def __init__(self):
        self.cantidad_trabajadores = 0
        self.cantidad_ordenes = 0
        self.ordenes = []
        self.conflictos_trabajadores = []
        self.ordenes_correlativas = []
        self.ordenes_conflictivas = []
        self.ordenes_repetitivas = []
        self.diccionario_indices = {}
        self.variables_totales=0
        self.tramos_salario=4
        self.valor_tramo=[-1_000, -1_200,-1_400,-1_500 ]
        self.dias_semana=6
        self.turnos=5
        self.variables=[]
        self.variables_binarias=[]
        self.variables_N=[]
        

    def load(self,filename):
        # Abrimos el archivo.
        f = open(filename)

        # Leemos la cantidad de trabajadores
        self.cantidad_trabajadores = int(f.readline())
        
        # Leemos la cantidad de ordenes
        self.cantidad_ordenes = int(f.readline())
        
        # Leemos cada una de las ordenes.
        self.ordenes = []
        for i in range(self.cantidad_ordenes):
            row = f.readline().split(' ')
            orden = Orden()
            orden.load(row)
            self.ordenes.append(orden)

        # Leemos la cantidad de conflictos entre los trabajadores
        cantidad_conflictos_trabajadores = int(f.readline())
        
        # Leemos los conflictos entre los trabajadores
        self.conflictos_trabajadores = []
        for i in range(cantidad_conflictos_trabajadores):
            row = f.readline().split(' ')
            self.conflictos_trabajadores.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes correlativas
        cantidad_ordenes_correlativas = int(f.readline())
        
        # Leemos las ordenes correlativas
        self.ordenes_correlativas = []
        for i in range(cantidad_ordenes_correlativas):
            row = f.readline().split(' ')
            self.ordenes_correlativas.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes conflictivas
        cantidad_ordenes_conflictivas = int(f.readline())
        
        # Leemos las ordenes conflictivas
        self.ordenes_conflictivas = []
        for i in range(cantidad_ordenes_conflictivas):
            row = f.readline().split(' ')
            self.ordenes_conflictivas.append(list(map(int,row)))
        
        
        # Leemos la cantidad de ordenes repetitivas
        cantidad_ordenes_repetitivas = int(f.readline())
        
        # Leemos las ordenes repetitivas
        self.ordenes_repetitivas = []
        for i in range(cantidad_ordenes_repetitivas):
            row = f.readline().split(' ')
            self.ordenes_repetitivas.append(list(map(int,row)))


        
        I=self.cantidad_trabajadores  #trabajador
        J=self.cantidad_ordenes  #orden o tarea
        K=self.dias_semana  #dia de semana
        L=self.turnos  #turno
        N=self.tramos_salario  #tramos
        W=N-1  #auxiliar activa tramos
        for i in range(I):  # i 
            for j in range(J):  # j 
                for k in range(K):  # k 
                    for l in range(L):  # l 
                        clave = f"X_{i}_{j}_{k}_{l}"  
                        valor = i * L*K*J + j * L*K + k * L + l  # Valor correlativo
                        self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for k in range(K):  # k 
                clave = f"D_{i}_{k}"
                valor = max_valor+1+i*K+k
                self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for j in range(J):  # j 
            for k in range(K):  # k 
                for l in range(L):  # l 
                    clave = f"R_{j}_{k}_{l}"  
                    valor = max_valor+1+ j * L*K + k * L + l  # Valor correlativo
                    self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for w in range(W):  # n  
                clave = f"W_{i}_{w}"
                valor = max_valor+1+i*W+w
                self.diccionario_indices[clave] = valor
                
        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for n in range(N):  # n  
                clave = f"N_{i}_{n}"
                valor = max_valor+1+i*N+n
                self.diccionario_indices[clave] = valor
        max_valor = max(self.diccionario_indices.values())

        self.variables=[0]* len(self.diccionario_indices)
        # Cerramos el archivo.
        f.close()

       

def get_instance_data():
    #file_location = sys.argv[1].strip()
    file_location='data/input_data.txt'
    instance = FieldWorkAssignment()
    instance.load(file_location)
    return instance
     


In [3]:
def genera_condiciones_variables():
    
    for key in data.diccionario_indices.keys():
        if key[0]=='R':
            for j in range(data.cantidad_ordenes):
                if key[2]==str(j):
                    posicion=data.diccionario_indices[key]
                    data.variables[posicion]=data.ordenes[j].beneficio
    for key in data.diccionario_indices.keys():
        if key[0]=='N':
            for numero_tramo in range(data.tramos_salario):
                if key[4]==str(numero_tramo):
                    posicion=data.diccionario_indices[key]
                    data.variables[posicion]=data.valor_tramo[numero_tramo]
    data.variables_N=data.variables[data.diccionario_indices['N_0_0']:]
    data.variables_binarias=data.variables[:data.diccionario_indices['N_0_0']]

In [4]:
# Obtenemos los datos de la instancia.
data = get_instance_data()
genera_condiciones_variables()
# Definimos el problema de cplex.
my_problem = cplex.Cplex()




In [5]:
def add_constraint_matrix_1(my_problem, data):
# R_k_l  <= a 1 para cada j  R_{j}_{k}_{l}
    for j in range(data.cantidad_ordenes):
    
        restriccion_indices=[]
        restriccion_valores=[]
        for t in data.diccionario_indices.keys():
            if t[0]=='R'and t[2]==str(j):
                
                restriccion_indices.append(data.diccionario_indices[t])
                restriccion_valores.append(1)
                
        indices = restriccion_indices
        values = restriccion_valores
        row = [indices,values]
        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])               

In [6]:
def add_constraint_matrix_2(my_problem, data):
# D_j-Sum(X_j_k_l)<=0 para todo i
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        if t[0]=='D'and t[2]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='X'and t[2]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])         


In [7]:
def add_constraint_matrix_3(my_problem, data):
#X_{i}_{j}_{k}_{l}
#Sum(X_j)<=1 para todo i,k,l
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):
                        for l in range(data.turnos):
                                restriccion_indices=[]
                                restriccion_valores=[]
                                for t in data.diccionario_indices.keys():
                                        if t[0]=='X'and t[2]==str(i) and t[6]==str(k) and t[8]==str(l):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(1)
                                
                                indices = restriccion_indices
                                values = restriccion_valores
                                row = [indices,values]
                                if len(indices)>0:
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])
                                else: 
                                        break
                        #                print(restriccion_indices)
                        #                print(restriccion_valores)
                        #                print(len(restriccion_valores))

In [8]:
def add_constraint_matrix_4(my_problem, data):
#X_{i}_{j}_{k}_{l}
#Sum(X_j_l)    <=4 para todo i,k
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):
                        
                        restriccion_indices=[]
                        restriccion_valores=[]
                        for t in data.diccionario_indices.keys():
                                if t[0]=='X'and t[2]==str(i) and t[6]==str(k):
                                        restriccion_indices.append(data.diccionario_indices[t])
                                        restriccion_valores.append(1)
                        
                        indices = restriccion_indices
                        values = restriccion_valores
                        row = [indices,values]
                        if len(indices)>0:
                                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[4])
                        else: 
                                break

In [9]:
def add_constraint_matrix_5(my_problem, data):
# D_k    <=5 para todo i
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        if t[0]=='D'and t[2]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                                             
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[5]) 
                        #                print(restriccion_indices)
                        #                print(restriccion_valores)
                        #                print(len(restriccion_valores))

In [10]:
#        I=self.cantidad_trabajadores  #trabajador
#        J=self.cantidad_ordenes  #orden o tarea
#        K=self.dias_semana  #dia de semana
#        L=self.turnos  #turno
#        N=self.tramos_salario  #tramos
#        W=N-1 
#data.cantidad_ordenes

In [11]:
def populate_by_row(my_problem, data):
#VERdef add_constraint_matrix_4(my_problem, data):

    # Definimos y agregamos las variables de la parte positiva de la formula beneficio
        coef_fun_objetivo_1 = data.variables_binarias
    
        my_problem.variables.add(
                obj = coef_fun_objetivo_1, 
                lb =[0]*len(data.variables_binarias), 
                ub =[1]*len(data.variables_binarias), 
                types=['B']*len(data.variables_binarias)
                ) 

        # Definimos y agregamos las variables de la parte negativa de la formula costo
        #cada trabajador tiene N1, N2, N3, N4  si el indice que les genera es posterior al resto esta justo!!!
        coef_fun_objetivo_2=data.variables_N

        my_problem.variables.add(
                obj = coef_fun_objetivo_2, 
                lb =[0]*len(data.variables_N), 
                ub =[5]*len(data.variables_N), 
                types=['I']*len(data.variables_N)
                ) 

        # Seteamos direccion del problema
        my_problem.objective.set_sense(my_problem.objective.sense.maximize)
        # ~ my_problem.objective.set_sense(my_problem.objective.sense.minimize)

        # Definimos las restricciones del modelo. Encapsulamos esto en una funcion. 
        add_constraint_matrix_1(my_problem, data)
        add_constraint_matrix_2(my_problem, data)
        add_constraint_matrix_3(my_problem, data)
        add_constraint_matrix_4(my_problem, data)
        add_constraint_matrix_5(my_problem, data)
        # Exportamos el LP cargado en myprob con formato .lp. 
        # Util para debug.
        my_problem.write('balanced_assignment.lp')

In [12]:

# Armamos el modelo.
populate_by_row(my_problem,data)

Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


In [14]:
def solve_lp(my_problem, data):
    
    # Resolvemos el ILP.
    
    my_problem.solve()

    # Obtenemos informacion de la solucion. Esto lo hacemos a traves de 'solution'. 
    x_variables = my_problem.solution.get_values()
    objective_value = my_problem.solution.get_objective_value()
    status = my_problem.solution.get_status()
    status_string = my_problem.solution.get_status_string(status_code = status)

    print('Funcion objetivo: ',objective_value)
    print('Status solucion: ',status_string,'(' + str(status) + ')')

    # Imprimimos las variables usadas.
    for i in range(len(x_variables)):
        # Tomamos esto como valor de tolerancia, por cuestiones numericas.
        if x_variables[i] > TOLERANCE:
            print('x_' + str(data.items[i].index) + ':' , x_variables[i])


Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


In [56]:
'''
def main():
    
    # Obtenemos los datos de la instancia.
    data = get_instance_data()
    
    genera_condiciones_variables()
    
    # Definimos el problema de cplex.
    my_problem = cplex.Cplex()
    
    # Armamos el modelo.
    populate_by_row(my_problem,data)

    # Resolvemos el modelo.
    solve_lp(my_problem,data)


#if __name__ == '__main__':
#    main()


"\ndef main():\n    \n    # Obtenemos los datos de la instancia.\n    data = get_instance_data()\n    \n    genera_condiciones_variables()\n    \n    # Definimos el problema de cplex.\n    my_problem = cplex.Cplex()\n    \n    # Armamos el modelo.\n    populate_by_row(my_problem,data)\n\n    # Resolvemos el modelo.\n    solve_lp(my_problem,data)\n\n\n#if __name__ == '__main__':\n#    main()\n"